In [1]:
import sys
sys.path.append("..")

In [2]:
from config import engine

In [3]:
import panel as pn
import pandas as pd
import hvplot.pandas

In [4]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

### Connect to Insight DB

### Get DBs

In [5]:
databases = pd.read_sql_query('select name from sql_databases', con=engine).name.to_list()

In [6]:
def get_last_months_logs(db_name, engine):
    today = datetime.utcnow()
    one_month_ago = today - relativedelta(months=1)
    query = f"""
SELECT
    id,
    name,
    status,
    log,
    timestamp
FROM status_log
WHERE name = '{db_name}'
AND timestamp >= '{one_month_ago.strftime('%Y-%m-%d')}'
ORDER BY timestamp 
"""
    df = pd.read_sql_query(sql=query, con=engine)
    df.timestamp = pd.to_datetime(df.timestamp)
    return df

### Get Last Month's logs

In [7]:
chart_title = 'Database Uptime'
db_selector_list = pn.widgets.Select(name='Databases', options=databases)

In [8]:
@pn.depends(db_selector_list)
def get_plot(db_selection):
    df = get_last_months_logs(db_selection, engine=engine)
    return df.hvplot.step(
        'timestamp', 
        'status', 
        grid=True, 
        ylim=(-1,2),
        yticks=[
            (0, 'OFFLINE'),
            (1, 'ONLINE'),
        ]
    )


In [9]:
pn.Row(pn.Column(chart_title, db_selector_list), get_plot).servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='Databases', options=['postgres_rds', ...], value='postgres_rds')
    [1] ParamFunction(function)

### Graph: Database Uptime

In [10]:
g1 = pn.Row(pn.Column(chart_title, db_selector_list), get_plot)

### Graph 2

g2 = pn.Row(pn.Column(chart_title, db_selector_list), get_plot)

### Graph 3

g3 = pn.Row(pn.Column(chart_title, db_selector_list), get_plot)

In [11]:
pn.serve({
    'database_uptime': g1, 
#     'david_graph_2': g2, 
#     'david_graph_3': g3
}, port=8070)

Launching server at http://localhost:8070
